Import

In [1]:
import os
import cv2
import numpy as np
from tqdm import tqdm

Read File

In [4]:
image_folder = 'your folder'

file_type = ".TIF"

save_folder = 'your folder'

img_path = [os.path.join(image_folder, path) for path in os.listdir(image_folder) if path.endswith(file_type)]
img_path.sort()
img_path = img_path[1100:2300]

if img_path:
    print(f"Find {len(img_path)} File")
else:
    print("No Find File")

Find 1200 File


Normal Rail

In [5]:
sum = cv2.imread(img_path[0])

for i in tqdm(img_path):
    n_img = cv2.imread(i)
    sum = np.maximum(n_img, sum)

cv2.imwrite(os.path.join(save_folder, f'output{file_type}'),sum)

100%|██████████| 1200/1200 [03:21<00:00,  5.95it/s]


True

Deflect Rail

In [6]:
scale = 1 - 2*(10**-3) # 縮放大小
freq = 5 # 相隔幾張照片進行縮放

sum = cv2.imread(img_path[0])
height, width = sum.shape[:2]
new_width = int(width * scale)
new_height = int(height * scale)

# 縮放偏移
x_offset = 0 # X軸靠左
y_offset = (height - new_height)//2 # Y軸置中

def zoom_out_with_padding(img):

    resized_img = cv2.resize(img, (new_width, new_height))
    canvas = np.zeros((height, width, 3), dtype=np.uint8)
    canvas[y_offset:y_offset+new_height, x_offset:x_offset+new_width] = resized_img
    
    return canvas

count = 0

for i in tqdm(img_path):
    count += 1

    if count % freq == 0:
        sum = zoom_out_with_padding(sum)

    n_img = cv2.imread(i)
    sum = np.maximum(n_img, sum)

cv2.imwrite(os.path.join(save_folder, f'output{file_type}'),sum)

100%|██████████| 1200/1200 [03:42<00:00,  5.38it/s]


True

Another Deflect Rail

In [17]:
# 縮放偏移
x_offset = 0.4 # 左側占比
y_offset = 1 # 上方占比

sum = cv2.imread(img_path[0])
height, width = sum.shape[:2]
sum = np.zeros((height, width, 3), dtype=np.uint8)

count = len(img_path)

for ind, picture in tqdm(enumerate(img_path), total=len(img_path)):

    n_img = cv2.imread(picture)
    new_height = max(1, (height * ind) // count)
    new_width = max(1, (width * ind) // count)

    resized_img = cv2.resize(n_img, (new_width, new_height))
    
    canvas = np.zeros((height, width, 3), dtype=np.uint8)

    less_height = height - new_height
    less_width = width - new_width
    h_start = int(less_height * y_offset)
    h_end =h_start + new_height
    w_start = int(less_width * x_offset)
    w_end = w_start + new_width

    canvas[h_start:h_end, w_start:w_end] = resized_img
    
    sum = np.maximum(canvas, sum)

cv2.imwrite(os.path.join(save_folder, f'output{file_type}'),sum)

100%|██████████| 1200/1200 [04:00<00:00,  4.98it/s]


True

Make Video

In [7]:
fps = 30

frame = cv2.imread(os.path.join(image_folder, img_path[0]))
# frame = cv2.rotate(frame, cv2.ROTATE_90_COUNTERCLOCKWISE)
height, width, layers = frame.shape

fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # 使用 mp4 編碼
video = cv2.VideoWriter(os.path.join(save_folder, 'output_video.mp4'), fourcc, fps, (width, height))

# 將每張圖片寫入影片
for image in tqdm(img_path):
    frame = cv2.imread(os.path.join(image_folder, image))
    # frame = cv2.rotate(frame, cv2.ROTATE_90_COUNTERCLOCKWISE)
    video.write(frame)

# 釋放影片寫入器
video.release()
print(f"影片已儲存")

100%|██████████| 1200/1200 [05:33<00:00,  3.60it/s]

影片已儲存
